<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CS152-Assignment-1:-The-8-puzzle" data-toc-modified-id="CS152-Assignment-1:-The-8-puzzle-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CS152 Assignment 1: The 8-puzzle</a></span></li><li><span><a href="#Extension-Questions" data-toc-modified-id="Extension-Questions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Extension Questions</a></span></li><li><span><a href="#
Question-1----
" data-toc-modified-id="
Question-1----
-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>
Question 1    
</a></span></li><li><span><a href="#
Question-2----
" data-toc-modified-id="
Question-2----
-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>
Question 2    
</a></span></li><li><span><a href="#
Question-3----
" data-toc-modified-id="
Question-3----
-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>
Question 3    
</a></span></li><li><span><a href="#Basic-Functionality-Tests" data-toc-modified-id="Basic-Functionality-Tests-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Basic Functionality Tests</a></span></li><li><span><a href="#Extension-Tests" data-toc-modified-id="Extension-Tests-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Extension Tests</a></span></li></ul></div>

<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

In [1]:
import numpy as np

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

In [2]:
def solvability_test(state):
    """
    Tests if a given input array is solvable based on the number of
    inversions on the board and the board size. Returns a boolean value.
    Solution adapted from: https://datawookie.netlify.com/blog/2019/04/
                           sliding-puzzle-solvable/
    With the extension rules from: https://www.cs.bham.ac.uk/~mdr/
                           teaching/modules04/java2/TilesSolvability.html
    
    Input:
        state (arr) Input state array as a list of lists.
        
    - If the grid width is odd, then the number of inversions in a
      solvable situation is even.
    - If the grid width is even, and the blank is on an even row counting
      from the bottom (second-last, fourth-last etc), then the number of
      inversions in a solvable situation is odd.
    - If the grid width is even, and the blank is on an odd row
      counting from the bottom (last, third-last, fifth-last etc)
      then the number of inversions in a solvable situation is even.
    """
    inversion_count = 0
    n = len(state)
    n_tiles = (len(state) ** 2)
    
    #reshaping the input to be of a valid format
    state = np.array(state).reshape(n, n) 
    
    zero_tile_loc = tuple(*np.argwhere(state == 0))
    tile_list = list(state.flatten())  

    for i in range(n_tiles - 1):
        for j in range(i+1, n_tiles):
            if tile_list[j] and tile_list[i] and tile_list[i] > tile_list[j]:
#                 print(tile_list[i], "is larger than", tile_list[j])
                inversion_count += 1
                
    # Different scenarios for solvability.
    if (n % 2 != 0) and (inversion_count % 2 == 0):
        return(True)
    elif (n % 2 == 0) and (zero_tile_loc[0] % 2 != 0) and (inversion_count % 2 != 0):
        return(True)
    elif (n % 2 == 0) and (zero_tile_loc[0] % 2 == 0) and (inversion_count % 2 == 0):
        return(True)
    else:
        return(False)

In [3]:
solvable_puzzle = [[5,7,6],[2,4,3],[8,1,0]]
solvability_test(solvable_puzzle)

True

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [4]:
class PuzzleNode():
    """
    Class PuzzleNode: Provides a structure for performing A* search
    for the n^2-1 puzzle.
    
    Attribute note: 
        Takes the state as any input shape. This is later converted
        to valid input (array) in the solvePuzzle function. 
    """
    def __init__(self, state, gcost = 0, fcost = 0, parent = None):

        self.gcost = gcost    # current best path cost g()
        self.fcost = fcost    # g() + estimated heuristic cost h()
        self.parent = parent  # parent for retrieving final solution
        self.state = state    # state of the puzzle at node n (arr)
        
        self.n = len(state)   # for print function
        self.pruned = False   # indicates if node can be ignored during search
    
    # CLASS FUNCTIONS
    # ---------------
    def __str__(self):
        """
        Returns the grid of the board state of a given node.
        """
        return('\n'.join([' '.join(['{:n}'.format(tile, str(self.n))
                for tile in row]) for row in self.state]))
    
    
    def __lt__(self, other):
        """
        Used in the solvePuzzle function for comparing node costs.
        """
        return self.fcost < other.fcost

In [5]:
test_state = [[0,1,2], [3,4,5], [6,7,8]]
test_state_unsolved = [[1,0,2], [3,4,5], [6,7,8]]
test_state_large = [[0,1,2,5], [3,4,5,5], [6,7,8,5], [2,3,4,5]]
test_state_faulty = [[0,1,2], [3,4,5,5], [6,7,8,5], [2,3,4,5]]
test_state_array = np.array([[0,1,2], [3,4,5], [6,7,8]])
test_state_unsolved_difficult = [[1,2,5], [3,4,0], [6,7,8]]

node = PuzzleNode(test_state_array)

In [6]:
print(node.__str__())
tuple(node.state.flatten())

0 1 2
3 4 5
6 7 8


(0, 1, 2, 3, 4, 5, 6, 7, 8)

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [7]:
# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles,
    given the board state (excluding 0 tile).
    
    Input:
        state (arr / lst) the board state as an n*n array. Converts the input
        if it is given as a list of lists. Required.
    """
    n = len(state)

    #reshaping the input to be of a valid format
    state = np.array(state).reshape(n,n)
    
    misplaced_tiles = [tile != i for i, tile in enumerate(state.flatten())]
    return(sum(misplaced_tiles)-1)


# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state,
    given the board state. It does so by flattening the array and then
    pairing each tile's coordinates with the "correct" coordinates. Then
    it calculates the distance between them.
    Adapted from: https://stackoverflow.com/questions/39759721/
                  calculating-the-manhattan-distance-in-the-eight-puzzle?rq=1
    
    Input:
        state (arr / lst) the board state as an n*n array. Converts the input
        if it is given as a list of lists. Required.
    """
    
    #reshaping the input to be of a valid format
    n_tiles = (len(state) ** 2)
    n = len(state)
    
    state = np.array(state).reshape(n, n)
        
    cur_state = list(state.flatten())  
    goal_state = list(range(n**2))
        
    manhattan_dist = [abs(b % n - g % n) + abs(b//n - g//n) for
                      b, g in ((cur_state.index(i), goal_state.index(i)) for
                      i in range(1, n_tiles))]
        
    return sum(manhattan_dist)
    
    
# Linear Conflict Heuristic
def h3(state):   # NOT IMPLEMENTED
    """
    This function returns the linear conflict score, a heuristic that
    dominates the Manhattan distance (it is always at least as large as the 
    Manhattan distance), given the board state.
    
    "Two tiles tj and tk are in a linear conflict if tj and tk are
    the same line, the goal positions of tj and tk are both in that
    line, tj is to the right of tk , and goal position of tj is to
    the left of the goal position of tk . Here line indicated both
    rows and columns. The linear conflict heuristic is calculated as
    Manhattan distance + 2*(Linear conflicts)."
    
    Code adapted from: https://stackoverflow.com/a/43967750

    Input:
        state: the board state as a list of lists        
    """
    linear_confl = 0
    
    # LINEAR CONFLICT ALGORITHM

    return h2(state) + 2 * linear_confl
    

heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [8]:
# Import any packages or define any helper functions you need here
from queue import PriorityQueue

# Main solvePuzzle function.
def solvePuzzle(state, heuristic=h1):
    """
    This function should solve the n**2-1 puzzle for any n > 2
    (although it may take too long for n > 4)).
    
    Inputs:
        state (list) The initial state of the puzzle as a list of lists.
        Required.
        heuristic (str) a handle to a heuristic function. Required.
        Options: [h1, h2, h3] (misplaced tiles, manhattan distance, XXX)
        
    Outputs:
        steps (int) The number of steps to optimally solve the puzzle, 
                excluding the initial state.
        expanded (int) The number of nodes expanded to reach the solution.
        max_frontier (int) The maximum size of the frontier over the search.
        opt_path (list) The optimal path as a list of list of lists.
                opt_path[:,:,i] gives a list of lists that represents
                the state of the board at step i of the solution.
        err (str) An error code.  If state is not of the appropriate size
                and dimension, return -1. If the state is not solvable,
                returns -2.
    """
    # INPUT TESTS
    # -----------
    #  To test for correct shape, we try to reshape the state from a list
    #  of lists to an array. To check if the input is valid (generally),
    #  we can then flatten & sort it & compare it to the goal state.
    #  If either test fails, the input is incorrect.
    # -----------
    #  Extension 1 - Initial state solvability XXX
    
    n = len(state)
    goal_state_flattened = list(range(n**2))

    if not solvability_test(state):
        print("This puzzle appears not solveable.")
        return(0, 0, 0, None, -2)
    
    try:
        state = np.array(state).reshape(n, n)
        # state is now a n*n array, no longer a list of lists!
    except:
        print("The input could not be shaped correctly.")
        return(0, 0, 0, None, -1)
    
    state_flattened = state.flatten()
    if sorted(state_flattened) != goal_state_flattened:
        print("Input invalid. Check your state for duplicates, please.")
        return(0, 0, 0, None, -1)
    
    
    # SEARCH SETUP
    # ------------
    #  Sets the initial node to the start state, initiates the dictionary
    #  for tracking search cost and generates the frontier.
    # ------------

    start_node = PuzzleNode(state, gcost = 0, fcost = heuristic(state))
    costs_db = {tuple(start_node.state.flatten()): start_node}

    # Frontier, stored as a Priority Queue to maintain ordering
    frontier = PriorityQueue()
    frontier.put(start_node)

    # other variables
    moves_orth = [(1,0),(0,1),(-1,0),(0,-1)]
    expansion_counter = 0
    frontier_max = 0
    
    
    # SEARCH ALGORITHM
    # ----------------
    #  Generates possible future moves and appends them to the frontier
    #  until a solution is found. The cost_bd keeps track of previously 
    #  explored states.
    # ----------------

    while not frontier.empty():
        
        # retrieving worst case frontier size
        if frontier.qsize() > frontier_max:
            frontier_max = frontier.qsize()

        cur_node = frontier.get()
        
        if cur_node.pruned:
            continue

        # Goal check
        if list(cur_node.state.flatten()) == goal_state_flattened:
            print("Algorithm terminated at goal state.")
            break
        
        # find 0 tile
        zero_tile = tuple(*np.argwhere(cur_node.state == 0))

        # expand in the orthogonal direction to generate sucessor states
        # avoiding states that would go outside the edges        
        next_moves = [tuple( [sum(i) for i in zip(zero_tile, move)] )
                      for move in moves_orth] 
        valid_moves = [i for i in next_moves if
                       (0 <= i[0] < n and 0 <= i[1] < n)]
        

        for m in valid_moves:
            gcost = cur_node.gcost + 1 # minimum cost for child
    
            # try the move by swapping 0 and non-0 tile
            test_state = np.copy(cur_node.state)
            z = zero_tile
            test_state[z], test_state[m] = test_state[m], test_state[z]        

            # If move is explored, update the path & mark for removal
            if tuple(test_state.flatten()) in costs_db:
                if costs_db[tuple(test_state.flatten())].gcost > gcost:
                    costs_db[tuple(test_state.flatten())].pruned = True
                else:
                    continue 
            
            # get heuristic cost from the node to the end and
            # add the child to the frontier & cost database
            hcost = heuristic(test_state) 
            next_node = PuzzleNode(test_state, gcost = gcost,
                        fcost = gcost + hcost,  parent=cur_node)
            
            frontier.put(next_node)
            costs_db[tuple(test_state.flatten())] = next_node 
            
        expansion_counter = expansion_counter + 1

    # RETRIEVING ALGORITHM INFO
    # -------------------------
    #  Reconstructs the path of the optimal solution and returns several
    #  summary statistics of the algorithm to evaluate performance.
    # -------------------------
    
    # reconstructing solution path
    opt_path_nodes = []
    solution_node = cur_node
    
    while solution_node:
        opt_path_nodes.append(solution_node)
        solution_node = solution_node.parent
        
    opt_path = []
    
    #reverses through the solution
    for j, node in reversed(list(enumerate(opt_path_nodes))):
        slot = len(opt_path_nodes) - 1 - j
        opt_path.append(node.state.tolist())
    
    return (len(opt_path_nodes)-1, expansion_counter,
            frontier_max, opt_path, 0)

In [9]:
working_puzzle = [[2,3,7],[1,8,0],[6,5,4]]

In [10]:
steps, exp, max_frontier, opt_path, err = solvePuzzle(working_puzzle)
print(steps, exp, max_frontier, err)

Algorithm terminated at goal state.
17 728 436 0


<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [11]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

Input invalid. Check your state for duplicates, please.


In [12]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [13]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


Algorithm terminated at goal state.
Algorithm terminated at goal state.
Algorithm terminated at goal state.
Algorithm terminated at goal state.
Algorithm terminated at goal state.
Algorithm terminated at goal state.


In [14]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

Algorithm terminated at goal state.
Algorithm terminated at goal state.


<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [15]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

This puzzle appears not solveable.


In [16]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

Algorithm terminated at goal state.
Algorithm terminated at goal state.
Algorithm terminated at goal state.
Algorithm terminated at goal state.
Algorithm terminated at goal state.
Algorithm terminated at goal state.


AssertionError: 

In [ ]:
## Memoization test - will be carried out after submission